In [58]:
from mistralai import Mistral
from tool_config import tools_conf
import json

### IMPORTE NECESSARY FUNCTIONS CALLED IN THE QUERY FUNCTION
# from .... import retrieve_best_productsV0
# from .... import retrieve_best_productsV1
from tools_func import *
# Retrieve the API key from the environment variable
api_key = "W7jZ5RO87zVxhO0gehFjjg0TqyXasmGj"
if not api_key:
    raise ValueError("MISTRAL_API_KEY environment variable not set.")
custom_agent_id = "ag:68495cb5:20241123:expert-tpmc:bbd4e63b"




class Agent:
    def __init__(self,tools_conf,names_to_functions):
        self.products = []
        self.client = Mistral(api_key=api_key)
        self.tools = tools_conf
        self.names_to_functions = names_to_functions
        self.agent_id = custom_agent_id

        self.user_query = ""
        self.data = ""
        self.advice = ""
        self.next_data_query = ""
        self.expert_input = ""
        self.querry_data = ""

    def get_response_to_user(self, user_message):
        self.user_query = user_message
        self.build_expert_input()
        while self.querry_data != "STOP":
            self.update_advice()
            self.get_next_querry()
            self.get_next_action()
            self.update_data(self.querry_data)
        return self.get_reseponse()

        
    def build_expert_input(self):
        self.expert_input = "{\"user_input\": \"" + self.user_query + "\"; \"data_available\": \"" + self.data + "\"; \"Advice\": \"" + self.advice + "\"; \"new_data_query\": \"" + self.next_data_query + "\"}"
        

    def update_advice(self):
        task = "based on the current input, update the advice to be made for the customer. Answer should be in the right format. Only the advice should be updated, and not the other fields"
        txt_message = task + "  " + self.expert_input
        messages = [
            {
                "role": "user",
                "content": txt_message
            }
        ]

        agent_response = self.client.agents.complete(agent_id=self.agent_id, messages=messages)
        self.expert_input = agent_response.choices[0].message.content

    def update_data(self, data):
        task = "based on the following, update the data available for the customer.data_vailable should now contain the information received in the querry. for example, if products are given, the list thousld be added to the available data.  Answer should be in the right format. "
        txt_message = task + "  " + data + "   " + self.expert_input
        messages = [
            {
                "role": "user",
                "content": txt_message
            }
        ]

        agent_response = self.client.agents.complete(agent_id=self.agent_id, messages=messages)
        self.expert_input = agent_response.choices[0].message.content
        
    def get_next_action(self):
        task = "based on the next_data_querry field of the current state, get the next action to be taken. Decide which of the tools to use to answer to the client the best way. Use the new_query_data information to get the right function to call, and the right parameters to use. Discard other fields. "
        txt_message = task + "  " + self.expert_input
        messages = [
            {
                "role": "user",
                "content": txt_message
            }
        ]
    
        
        response = self.client.chat.complete(
                model = "mistral-small-latest",
                messages = messages,
                tools = self.tools,
                tool_choice = "any",
                )
        function_name = response.choices[0].message.tool_calls[0].function.name
        function_params = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
        print(function_name)
        print(function_params)
        self.querry_data = self.names_to_functions[function_name](**function_params)
        
    
    def get_next_querry(self):
        task = "based on the current input, get the next data query to be made. Use actions, verbs, to indicate what should be querried, such as Find... Indentify... . If we already have the needed information, the data querry can be empty, 'no more information needed'. Take into account all the available data and advie. Dont ask more than needed. You should stop at some point. Only look for information that you don't alreadt have. Answer should be in the right format. "
        txt_message = task + "  " + self.expert_input
        messages = [
            {
                "role": "user",
                "content": txt_message
            }
        ]

        agent_response = self.client.agents.complete(agent_id=self.agent_id, messages=messages)
        self.expert_input = agent_response.choices[0].message.content

    def get_reseponse(self):
        task = "It seems all the information that could be obtained from the data base is in your possessing. Based on the current input, get the response to be made to the customer. Answer should be a string understandable by the client. Data querry should now be empty, as the information is transfered from there to available data. "
        txt_message = task + "  " + self.expert_input
        messages = [
            {
                "role": "user",
                "content": txt_message
            }
        ]

        agent_response = self.client.agents.complete(agent_id=self.agent_id, messages=messages)
        self.expert_input = agent_response.choices[0].message.content
        return self.expert_input

In [71]:
def no_more_info_needed():
    return "STOP"


tools_conf = [
    {
        "type": "function",
        "function": {
            "name" : "no_more_info_needed",
            "description": "If no more information is needed, call this function, to stop the process.",
            "parameters": {}
            }
    },
    {
        "type": "function",
        "function": {
            "name": "get_k_nearests_product",
            "description": "Search for the k products closest to the user's query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query string to search for similar products."
                    },
                    "k": {
                        "type": "integer",
                        "description": "The number of nearest products to retrieve."
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_k_nearest_users",
            "description": "Search for users who have a similar profile to the target user.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_id": {
                        "type": "integer",
                        "description": "The unique identifier for the user."
                    },
                    "k": {
                        "type": "integer",
                        "description": "The number of nearest users to retrieve."
                    }
                },
                "required": ["user_id", "k"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_best_purchases_from_neighbours",
            "description": "Retrieve the best products purchased by users with similar profiles.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_id": {
                        "type": "integer",
                        "description": "The unique identifier for the user."
                    }
                },
                "required": ["user_id"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_not_delivered",
            "description": "Returns a text summarizing the undelivered purchases of the user.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_id": {
                        "type": "integer",
                        "description": "The unique identifier for the user."
                    }
                },
                "required": ["user_id"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "handle_insufficient_info",
            "description": "Handles cases where there is not enough information to proceed with a product search and requests additional details from the user.",
            "parameters": {
                "type": "object",
                "properties": {
                    "missing_fields": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "The specific fields that are missing from the input, requiring user input."
                    },
                    "suggested_fields": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "Fields that are most critical to gather additional information for a search."
                    },
                    "action": {
                        "type": "string",
                        "enum": ["request_info", "abort"],
                        "description": "Action to be taken: request more information or abort the process."
                    }
                },
                "required": ["missing_fields", "action"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "check_user_purchase_history",
            "description": "Analyzes user purchase history and current orders to identify relevant patterns or preferences for product recommendations.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_id": {
                        "type": "integer",
                        "description": "The unique identifier for the user whose purchase history is being analyzed."
                    },
                    "k": {
                        "type": "integer",
                        "description": "How many purchases to recover."
                    }
                },
                "required": ["user_id", "k"],
                "additionalProperties": False
            }
        }
    }
]


In [60]:
# fichier des function appellé par query 
from User.db_utilitaries import RetrieveDatabase, UpdateDatabase
from RAG_product.Faiss_database import FaissDatabase
from Product.utils_db_product import Retrieve_from_db_prd
from basemistral import BaseMistral

from typing import Tuple

def no_more_info_needed():
    return "STOP"

def get_k_purchase(user_id:int, k:int = 5)-> str:
    """recupère les k dernière purchases dans la database

    Args:
        user_id (int): 
        k (int, optional):  Defaults to 5.

    Returns:
        str
        """
    user_id = 0 #set pour la demonstration
    res = ""
    purchases = RetrieveDatabase.get_last_k_purchases_by_user_id(user_id, k )
    for purchase in purchases:
        res += RetrieveDatabase.tuple_to_detailed_text(purchase)
        res += '\n' 
    if not res:
        res = "NO HISTORY"
    return res 



def get_k_nearests_product(query: str, k :int = 3) -> str :
    """Use RAG to get the 5 nearest product from the query , based on similarity with the descriptions 

    Args:
        query (str): 
        k (int, optional): Defaults to 3.
    """
    id_produits_list = FaissDatabase.search(query, k)
    produits = Retrieve_from_db_prd.get_product_by_id_list(id_produits_list)
    products_refined = [(produit[1], produit[2], produit[4]) for produit in produits]
    
    
    #Appeler la fonction qui affiche les produits
    
    #on va résumer les infos, donc pour ca on appelle baseMistral
    model = BaseMistral()
    text = ''
    i=0
    for produit in produits:
        text_produit = Retrieve_from_db_prd.product_to_paragraph(produit)
        text += f"Produit {i}: \n "
        text += model.summarize(text_produit)
        text += '\n'
        i+=1
        
    return text, products_refined



def get_k_nearest_users( user_id : int , k= 3) -> list[Tuple]:
    """ utilise faiss pour rechercher les k users les plus proches en ce basant sur le texte d'information

    Args:
        user_id (int)

    Returns:
        list[Tuple]
    """
    path_db_users = "Manager/User/userDB/user_index_faiss.faiss"
    user_id = 0 # pour le test
    
    user_info = RetrieveDatabase.get_user_info(user_id)
    
    index_neighbours = FaissDatabase.search(user_info, k, db_path= path_db_users)
    
    users = []
    for i in range(len(index_neighbours)):
        if index_neighbours[i] == user_id:
            index_neighbours.pop(i)
        else:
            users.append(RetrieveDatabase.get_user_by_id(index_neighbours[i]))
    return index_neighbours, users 


def get_best_purchases_from_neighbours(user_id = 0) -> str:
    """ Renvoie les 3 meilleurs achats des  3 voisins les plus proches, en version texte , cf function 

    Args:
        user_id (int, optional): _description_. Defaults to 0.
    """
    user_id = 0 
    index_neighbours, _  = get_k_nearest_users(user_id, 3)
    purchases =  []
    for i in index_neighbours:
        neighbours_id = index_neighbours[i]
        purchase_neighbours = RetrieveDatabase.get_best_k_purchases_by_user_id(neighbours_id, 3)
        purchases.extend(purchase_neighbours)
    text = ""
    for purchase in purchases : 
        text += RetrieveDatabase.tuple_to_detailed_text(purchase)
        text += "\n"
        
    return text 



def get_not_delivered(user_id: int) -> str:
    """renvoie un texte résumant les purchases non délivrer du user

    Args:
        user_id (int)
    """
    purchases = RetrieveDatabase.get_not_delivered_purchases(user_id)
    text = "Here are the commands not received: \n"
    
    for purchase in purchases:
        product_name, purchase_date  = purchase[2], purchase[3]
        text += f"{product_name}, purchased on the {product_name}"
        
    return text



def cancel_purchase(user_id: int,
                      purchase_id: int = None,
                      product_name: str = None,
                      purchased_date = None) -> str:
    """cancel order

    Args:
        user_id (int): 
        purchase_id (int, optional):. Defaults to None.
        product_name (str, optional): Defaults to None.
        purchased_date (_type_, optional):  Defaults to None.

    Returns:
        str: _description_
    """
    res = UpdateDatabase.suppress_purchase(user_id, purchase_id, product_name, purchased_date)
    return res 

names_to_functions = {
    "get_k_purchase": get_k_purchase,
    "get_k_nearests_product": get_k_nearests_product,
    "get_k_nearest_users": get_k_nearest_users,
    "get_best_purchases_from_neighbours": get_best_purchases_from_neighbours,
    "get_not_delivered": get_not_delivered,
    "cancel_purchase": cancel_purchase,
    "no_more_info_needed": no_more_info_needed
}



    

    

In [61]:
agent = Agent(tools_conf=tools_conf, names_to_functions=names_to_functions)


In [62]:
agent.user_query = "I have dry skin"
agent.build_expert_input()
print(agent.expert_input)
agent.update_advice()
print(agent.expert_input)
agent.get_next_querry()
print(agent.expert_input)

{"user_input": "I have dry skin"; "data_available": ""; "Advice": ""; "new_data_query": ""}
```json
{"user_input": "I have dry skin"; "data_available": ""; "Advice": "Use a gentle, hydrating cleanser and apply a rich moisturizer twice a day."; "new_data_query": ""}
```
```json
{"user_input": "I have dry skin"; "data_available": ""; "Advice": "Use a gentle, hydrating cleanser and apply a rich moisturizer twice a day."; "new_data_query": "Identify moisturizers suitable for dry skin."}
```


In [63]:
agent.get_next_action()


get_k_nearests_product
{'query': 'moisturizers suitable for dry skin'}
[694, 74, 491]
C:\Users\monce\Desktop\HackQB\HackathonQuantumX\Manager\Product\products.db
I AM HERE


In [64]:
print(agent.querry_data[0])

Produit 0: 
 The 'DND DC 103 Bamboo Brown Gel & Matching Polish Set - DND DC Gel & Lacquer' is priced at $8.31 and has an average rating of 4.0 out of 5. This product follows a professional 3-step gel system, requiring polishes to be cured under a LED lamp with DC Base Gel and DC Top Gel. Nail lacquers need to be air-dried. The application process involves five steps: applying a single coat of DND base gel and curing for 30 seconds, applying a single layer of gel color and curing for 30-45 seconds, repeating the gel color application and curing, applying DND top coat and curing, and ensuring nails are hydrated daily. Thick layers of gel color can result in uneven curing, leading to chipping and peeling.
Produit 1: 
 The 'ANOGOL Hair Cap+Lace Front Platinum Blonde Wig Synthetic Hair Wigs Long Curly Blonde Wig' is priced at $46.99 and has an average rating of 3.0 out of 5. This synthetic hair wig is designed for women and is suitable for daily life, movies, cosplay, dress-up, and Hallowe

In [68]:
agent.update_data(agent.querry_data[0])

In [69]:
print(agent.expert_input)

```json
{
  "user_query": "I have dry skin",
  "data_available": "Client has dry skin. Products: ['DND DC 103 Bamboo Brown Gel & Matching Polish Set - DND DC Gel & Lacquer', 'ANOGOL Hair Cap+Lace Front Platinum Blonde Wig Synthetic Hair Wigs Long Curly Blonde Wig', 'NYC New York Color HD Waterproof Automatic Eyeliner - #004 Turquoise']",
  "Advice": "Use a gentle, hydrating cleanser and apply a rich moisturizer twice a day. Consider products like the 'DND DC 103 Bamboo Brown Gel & Matching Polish Set - DND DC Gel & Lacquer' for nail care and the 'ANOGOL Hair Cap+Lace Front Platinum Blonde Wig Synthetic Hair Wigs Long Curly Blonde Wig' for hair care.",
  "data_query": "Identify moisturizers suitable for dry skin and nail care products."
}
```


In [70]:
agent.get_next_querry()
print(agent.expert_input)

```json
{
  "user_query": "I have dry skin",
  "data_available": "Client has dry skin. Products: ['DND DC 103 Bamboo Brown Gel & Matching Polish Set - DND DC Gel & Lacquer', 'ANOGOL Hair Cap+Lace Front Platinum Blonde Wig Synthetic Hair Wigs Long Curly Blonde Wig', 'NYC New York Color HD Waterproof Automatic Eyeliner - #004 Turquoise']",
  "Advice": "Use a gentle, hydrating cleanser and apply a rich moisturizer twice a day. Consider products like the 'DND DC 103 Bamboo Brown Gel & Matching Polish Set - DND DC Gel & Lacquer' for nail care and the 'ANOGOL Hair Cap+Lace Front Platinum Blonde Wig Synthetic Hair Wigs Long Curly Blonde Wig' for hair care.",
  "data_query": "no more information needed"
}
```


In [57]:
agent.get_next_action()

get_k_nearests_product
{'k': 3, 'query': 'dry skin care products'}
[694, 491, 36]
C:\Users\monce\Desktop\HackQB\HackathonQuantumX\Manager\Product\products.db
I AM HERE
